In [56]:
def MakeInteractiveSeparationContrastPlotOfNearbyRVPlanets(orbits, plotx, ploty, phases, 
                                                           saveplot = True, 
                                                           filt = 'None', xaxis_label = '',
                                                           annotation_text = '', IWA = 2,
                                                           ytop = 2e-6, ybottom = 2e-10,
                                                           xright = 20, xleft = 0,
                                                           ncolors = 10, ticklocs = 'None', ticklabels = 'None',
                                                          output_file_name = 'RVPlanetContrastPlot'):


    rad = orbits['Re'].copy()
    spt = orbits['SpT Number'].copy()
    
    plotx, ploty = np.array(plotx),np.array(ploty)
    multiplier = 2
    datadf = pd.DataFrame(data={'plotx':plotx, 'ploty':ploty, 'color':spt, 'markersize':rad*multiplier,
                               'name':orbits['pl_name'], 'rad':rad, 'spt':spt, 'dist':orbits['sy_dist'],
                                'phases':phases, 'plotx_og':plotx, 'ploty_og':ploty, 'iwa': 2
                               })
    datadict = datadf.to_dict(orient = 'list')

    from bokeh.plotting import figure, show, output_file, save
    from bokeh.io import output_notebook
    from bokeh.models import LinearColorMapper, ColumnDataSource, LinearInterpolator
    from bokeh.models import  Range1d, LabelSet, Label, ColorBar, FixedTicker, Span
    from bokeh.models import CustomJS, Slider
    from bokeh.layouts import column, row
    from bokeh.palettes import Magma256, Turbo256, brewer
    from bokeh.transform import linear_cmap
    #output_notebook()


    data=ColumnDataSource(data=datadict)


    tools = "hover, zoom_in, zoom_out, save, undo, redo, reset, pan"
    tooltips = [
        ('Planet', '@name'),
        #("(x,y)", "($x, $y)"),
        ('Cont', '@ploty'),
        ('Phase [deg]', '@phases{0}'),
        ('Sep [lod]', '@plotx{0.00}'),
        ('Rad [Rearth]','@rad{0.00}'),
        ('SpT','@spt{0.0}'),
        ('Dist [pc]','@dist{0.0}')
    ]

    p = figure(width=900, height=750, y_axis_type="log", tools=tools, 
               tooltips=tooltips, toolbar_location="above")

#     mapper = linear_cmap(field_name='color', 
#                          #palette=Magma256,
#                          palette=Turbo256[::-1],
#                          low=min(spt), high=max(spt),
#                         low_color=Turbo256[::-1][150], high_color=Turbo256[::-1][200])
    
    mapper = linear_cmap(field_name='phases', 
                         palette=brewer['PuOr'][ncolors],
                         #palette=Magma256,
                         #palette=Turbo256[::-1],
                         #low=min(phases), high=max(phases))
                         low=20, high=150)
    
    p.circle('plotx','ploty', source=data, fill_alpha=0.8, size='markersize', 
             line_color=mapper, color=mapper)

    
    color_bar = ColorBar(color_mapper=mapper['transform'], width=15, 
                         location=(0,0), title="Phase",
                        title_text_font_size = '20pt',
                         major_label_text_font_size = '15pt')
    
    if ticklocs == 'None':
        ticklocs = np.arange(0,180,20)
        ticklabels = {}
        [ticklabels.update({ticklocs[i]:str(np.round(ticklocs[i],decimals=-1)).replace('.0','')}) for i in range(len(ticklocs))]
    color_bar.ticker=FixedTicker(ticks=ticklocs)
    color_bar.major_label_overrides = ticklabels

    p.add_layout(color_bar, 'right')

    label = Label(
        text= annotation_text,
        x=50, y=20,
        x_units="screen", y_units="screen",text_font_size = '20pt'
    )
    p.add_layout(label)
    
    delt = np.log10(ytop) - np.log10(ybottom)
    
    x,y = 16, 10**(np.log10(ybottom) + (0.9*delt))
    p.circle(x,y, fill_alpha=0.6, size=11*multiplier,
             color='black')
    label1 = Label(x=x, y=y, text=r'\[ 11 R_\oplus\]',
                       x_offset=20, y_offset=-20,
                       text_font_size = '20pt')
    p.add_layout(label1)
    
    x,y = 16, 10**(np.log10(ybottom) + (0.85*delt))
    p.circle(x,y, fill_alpha=0.6, size=5*multiplier, 
             color='black')
    label2 = Label(x=x, y=y, text=r'$$5 R_\oplus$$',
                      x_offset=20, y_offset=-20,text_font_size = '20pt')
    p.add_layout(label2)
    x,y = 16, 10**(np.log10(ybottom) + (0.8*delt))
    p.circle(x,y, fill_alpha=0.6, size=1*multiplier, 
             color='black')
    label3 = Label(x=x, y=y, text=r'$$1 R_\oplus$$',
                      x_offset=20, y_offset=-20,text_font_size = '20pt')
    p.add_layout(label3)

    p.xaxis.axis_label = xaxis_label
    p.yaxis.axis_label = r'\[ \mathrm{Planet/Star\; Reflected\; Light\; Flux\; Ratio} \]'
    p.xaxis.axis_label_text_font_size = '20pt'
    p.yaxis.axis_label_text_font_size = '20pt'
    p.yaxis.major_label_text_font_size = "15pt"
    p.xaxis.major_label_text_font_size = "15pt"
    
    iwa = Span(location=IWA,
                              dimension='height', line_color='grey',
                              line_dash='dashed', line_width=3)
#     iwa = Span(location=data['iwa'],
#                                dimension='height', line_color='grey',
#                                line_dash='dashed', line_width=3)
    p.add_layout(iwa)
    
    p.x_range=Range1d(xleft,xright)
    p.y_range=Range1d(ybottom,ytop)
    

    AgSlider = Slider(start=0.05, end=1.0, value=0.3, step=.01, title="Geometric Albedo")
    IWASlider = Slider(start=1, end=10, value=2, step=.5, title="IWA")
    LambdaSlider = Slider(start=400, end=2000, value=800, step=50, title="Wavelength [nm]")
    DSlider = Slider(start=2, end=34, value=25.4, step=0.5, title="Primary Mirror Diameter [m]")
#         
#        var newx = x.map(d => d * 6.3/newlod );
    sliders_callback_code = """
        var Ag = Ag.value;
        var Lambda = Lambda.value;
        var D = D.value;
        
        var lod = 6.3;
        var newlod = ((Lambda/1000) / D) * 1000
        
        var y = source.data['ploty_og'];
        var x = source.data['plotx_og'];
        var newy = y.map(m => m * Ag/0.3 );
        var newx = x.map(b => b * 800/Lambda );
        var newx = newx.map(d => d * D/25.4 );


        console.log(newy)
        console.log(newx)
        source.data['ploty'] = newy;
        source.data['plotx'] = newx;
        source.change.emit();
    """

    slider_args = dict(source=data, Ag=AgSlider, Lambda=LambdaSlider, D=DSlider)
    
    AgSlider.js_on_change('value', CustomJS(args=slider_args,code=sliders_callback_code))
    LambdaSlider.js_on_change('value', CustomJS(args=slider_args,code=sliders_callback_code))
    DSlider.js_on_change('value', CustomJS(args=slider_args,code=sliders_callback_code))


    #show(row(p, column(AgSlider)))
    #show(column(p, row(AgSlider),row(LambdaSlider),row(DSlider)))
    output_file(output_file_name+".html")
    save(column(p, row(AgSlider),row(LambdaSlider),row(DSlider)))

    
#     if saveplot:
#         output_file(output_file_name+".html")
#         save(p)
#     else:
#         show(p)
#         pass

    return p

orbits = pd.read_csv('GMagAO-X-RV-targets-with-orbital-params.csv')
MakeInteractiveSeparationContrastPlotOfNearbyRVPlanets(orbits, 
                                                       np.array(orbits['separation_at_average_separation']),
                                                       np.array(orbits['contrast_at_average_separation']), 
                                                       np.array(orbits['phase_at_average_separation']),
                                                       filt = "i'",
                                                       xaxis_label = r'\[ \mathrm{Avg\; Projected\; Separation}\; [\lambda/D]\]',
                                                       #annotation_text = r"$$\mathrm{For}\; i'\; filter, \;D = 25.4m\$$",
                                                       annotation_text = '',
                                                       saveplot = False,
                                                      output_file_name = 'RVPlanets-Average-separation-contrast')

figure(id='p33145', ...)

In [57]:
len(orbits)

410

In [60]:
np.max(orbits['sy_dist'])

69.8683

In [61]:
ticks = np.arange(0,150,20)
ticklabels = {}
[ticklabels.update({ticks[i]:str(np.round(ticks[i],decimals=-1)).replace('.0','')}) for i in range(len(ticks))]
orbits = pd.read_csv('GMagAO-X-RV-targets-with-orbital-params.csv')
MakeInteractiveSeparationContrastPlotOfNearbyRVPlanets(orbits, 
                                                       orbits['separation_at_highest_contrast'],
                                                       orbits['contrast_at_highest_contrast'], 
                                                       orbits['phase_at_highest_contrast'],
                                                       filt = "i'",
                                                       xaxis_label = r'\[ \mathrm{Avg\; Projected\; Separation}\; [\lambda/D]\]',
                                                       #annotation_text = r"$$\mathrm{For}\; i'\; filter, \;D = 25.4m\$$",
                                                       annotation_text = '',
                                                       saveplot = False,
                                                       ncolors=8, ticklocs = ticks, ticklabels = ticklabels,
                                                       ytop = 2e-2, ybottom = 5e-8,
                                                       xright = 4, xleft = 0,
                                                      output_file_name = 'RVPlanets-separation-highest-contrast')

/var/folders/tm/1xdh6wyn1qv7mhy_c10d5c_w0000gn/T/ipykernel_2673/111330036.py:73: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if ticklocs == 'None':


figure(id='p33405', ...)

In [62]:
orbits = pd.read_csv('GMagAO-X-RV-targets-with-orbital-params.csv')
MakeInteractiveSeparationContrastPlotOfNearbyRVPlanets(orbits, 
                                                       orbits['separation_at_largest_separation'],
                                                       orbits['contrast_at_largest_separation'], 
                                                       orbits['phase_at_largest_separation'],
                                                       filt = "i'",
                                                       xaxis_label = r'\[ \mathrm{Avg\; Projected\; Separation}\; [\lambda/D]\]',
                                                       #annotation_text = r"$$\mathrm{For}\; i'\; filter, \;D = 25.4m\$$",
                                                       annotation_text = '',
                                                       saveplot = False,
                                                      output_file_name = 'RVPlanets-Max-separation-contrast')

figure(id='p33665', ...)

In [68]:
len(np.where(np.isnan(orbits['pl_orbincl']))[0])

389